# Bus Maintenance

In [ ]:
import gurobipy as grb
import pandas as pd
import numpy as np
np.set_printoptions(threshold=np.inf)

import scipy.sparse as spr
import os

In [ ]:
nbX = 10
nbT = 40
nbY = 2

## Solution with linear programming

In [ ]:
IdX = spr.identity(nbX)
LX = spr.dia_matrix(np.roll(IdX.toarray(),1,axis=1))
RX = spr.coo_matrix(np.concatenate((np.ones((nbX,1)),np.zeros((nbX,nbX-1))), axis =1))
P = spr.kron(np.array([[1,0]]).T, (0.75 * IdX + 0.25 * LX)) + spr.kron(np.array([[0,1]]).T, RX)
#P.toarray()

In [ ]:
IdT = spr.identity(nbT)
NT_temp = np.roll(IdT.toarray(),1,axis=1)
NT_temp[nbT-1]=0
NT = spr.dia_matrix(NT_temp)
A = spr.kron(spr.kron(IdT, np.array([[1,1]]).T), IdX) - spr.kron(NT, P)
#A.toarray()

In [ ]:
overhaulCost = 8000
maintCost = lambda x: x * 500
beta = 0.9
discount = lambda x: beta**x

In [ ]:
n1_x = np.repeat(1, nbX)
u_xy = np.concatenate((-np.asarray(list(map(maintCost,range(1,nbX)))), np.repeat(-overhaulCost, nbX+1)), axis = None)
u_xyt = spr.kron(np.asarray(list(map(discount,range(1,nbT+1)))),u_xy)

b_xt = np.concatenate((n1_x, np.repeat(0,int(nbX*(nbT-1)))),axis =None)

In [ ]:
m = grb.Model('BusMaintenance')
xt = m.addVars(nbT,nbX, obj=b_xt, name='xt', lb=float('-inf'))
m.update()
for i in range(A.shape[0]):
    L = grb.LinExpr(A[i,:].toarray().T,xt.select('*'))
    m.addConstr(L,'>',u_xyt.toarray()[0][i])

In [ ]:
m.optimize()
if m.status == grb.GRB.Status.OPTIMAL:
    solution = m.getAttr('x', xt)
    for state in range(nbX):
        print(solution[(state, 0)])

## Backward Induction

In [ ]:
U_x_t = np.zeros((nbX,nbT))

u_xyT = u_xyt.toarray().reshape((nbT, nbY, nbX))
u_xyT[nbT-1,:,:].T

contVals = np.max(u_xyT[nbT-1,:,:].T, axis = 1)
U_x_t[:,nbT-1] = contVals

In [ ]:
for t in reversed(range(nbT-1)):
    
    myopic = u_xyt.toarray().reshape((nbT, nbY, nbX))[t,:,:].T
    Econtvals = P.dot(contVals).reshape(nbY, nbX).T
    
    contVals = np.max(myopic+Econtvals, axis = 1)
    U_x_t[:,t] = contVals